In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-8b-instant"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/digital-support-engineer/job/R-57421")
page_data = loader.load().pop().page_content
print(page_data)






















Digital Support Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



S

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Digital Support Engineer',
 'experience': '2 years of experience in the job offered or in an engineering-related occupation',
 'skills': ['Oracle Database and querying',
  'Sterling Commerce',
  'Microservices',
  'APIs',
  'Splunk',
  'NewRelic',
  'Signal FX',
  'Powershell scripts',
  'Website and application troubleshooting',
  'Log analysis',
  'Alert creation'],
 'description': 'Partner with internal teams and external suppliers to deliver outstanding technical IT support for front-end apps platforms along with back-end tools and processes like pricing, order fulfillment, and supply chain; collaborate with teams all around the world in a follow-the-sun model, focusing on our North America geography, to support our digital capabilities and to resolve complex and critical issues; develop launches of new and high heat product; participate in execution of live ops initiatives and promotion deployments and resolve order management issues; participate in global projects that 

In [9]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}

In [18]:
job

{'role': 'Digital Support Engineer',
 'experience': '2 years of experience in the job offered or in an engineering-related occupation',
 'skills': ['Oracle Database and querying',
  'Sterling Commerce',
  'Microservices',
  'APIs',
  'Splunk',
  'NewRelic',
  'Signal FX',
  'Powershell scripts',
  'Website and application troubleshooting',
  'Log analysis',
  'Alert creation'],
 'description': 'Partner with internal teams and external suppliers to deliver outstanding technical IT support for front-end apps platforms along with back-end tools and processes like pricing, order fulfillment, and supply chain; collaborate with teams all around the world in a follow-the-sun model, focusing on our North America geography, to support our digital capabilities and to resolve complex and critical issues; develop launches of new and high heat product; participate in execution of live ops initiatives and promotion deployments and resolve order management issues; participate in global projects that 

In [16]:
job= json_res
job['skills']

['Oracle Database and querying',
 'Sterling Commerce',
 'Microservices',
 'APIs',
 'Splunk',
 'NewRelic',
 'Signal FX',
 'Powershell scripts',
 'Website and application troubleshooting',
 'Log analysis',
 'Alert creation']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Digital Potential with AtliQ

Dear [Client Name],

I came across your job description for a Digital Support Engineer and was impressed by the scope of responsibilities and the skills required for the role. At AtliQ, we specialize in facilitating seamless integration of business processes through automated tools, and I believe our expertise can help you fulfill your needs.

As a Digital Support Engineer at AtliQ, you will be working with our team of experts to deliver outstanding technical IT support for front-end apps platforms and back-end tools and processes. Our team has extensive experience in Oracle Database and querying, Sterling Commerce, Microservices, APIs, Splunk, NewRelic, Signal FX, and Powershell scripts, which aligns perfectly with your job requirements.

AtliQ has a proven track record of delivering successful projects across various industries. Some of our notable projects include:

* Java-based solutions: [https://example.com/java-portfolio](https: